In [1]:
import os
cwd = os.getcwd()
cwd

'/Users/clarisa/Documents/fast_ai/course-v3-master/nbs/dl2'

In [2]:
from IPython.display import Image

In [3]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [4]:
#export
from exp.nb_01 import *

def test(a,b,cmp,cname=None):
    if cname is None: cname=cmp.__name__
    assert cmp(a,b),f"{cname}:\n{a}\n{b}"

def near(a,b): 
    return torch.allclose(a, b, rtol=1e-3, atol=1e-5)

def test_near(a,b): 
    test(a,b,near)
    

def get_data():
    """
    Loads the MNIST data from before
    """
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): 
    """
    Normalizes an input array
    Subtract the mean and divide by standard dev
    result should be mean 0, std 1
    """
    return (x-m)/s

def test_near_zero(a,tol=1e-3): 
    assert a.abs()<tol, f"Near zero: {a}"

In [5]:
x_train, y_train, x_valid, y_valid = get_data()

In [6]:
train_mean,train_std = x_train.mean(),x_train.std()

In [7]:
print("original mean and std dev: ", train_mean, train_std)

original mean and std dev:  tensor(0.1304) tensor(0.3073)


Normalize the stuff:

In [8]:
# Image(cwd + "/images/" + "9a2.png", width = 500)

In [9]:
x_train = normalize(x_train,train_mean, train_std)

In [10]:
x_valid = normalize(x_valid,train_mean,train_std)

In [11]:
train_mean,train_std = x_train.mean(), x_train.std()
print("Normalized mean and std dev: ",train_mean,train_std)

Normalized mean and std dev:  tensor(0.0001) tensor(1.)


In [12]:
test_near_zero(train_mean)

In [13]:
test_near_zero(train_std - 1)

In [14]:
tr,tc = x_train.shape
tr,tc

(50000, 784)

In [15]:
y_train.max(),y_train.min()

(tensor(9), tensor(0))

In [16]:
classes = y_train.max() + 1;
classes

tensor(10)

I am in some pain due to the amount of BS happening in this data set...

Our first model will have 50 hidden units blah blah blah

In [17]:
def lin(x, w, b):
    return x@w + b

In [18]:
#number of hidden units:
nh = 50

In [19]:
w1 = torch.randn(tc,nh)/math.sqrt(tc);
w1.shape

torch.Size([784, 50])

In [20]:
b1 = torch.zeros(nh)

In [21]:
#For simplicity, we will make our model output 1 number instead\
#of outputting a vector.shape == classes
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [22]:
def relu(x): return x.clamp_min(0.)

In [23]:
ccl = torch.randn(10);
ccl

tensor([ 0.4929,  1.7259,  0.6689, -0.0582,  1.6048,  0.3592, -0.4413, -1.4329,
        -1.1167, -1.4565])

In [24]:
ccl.clamp_min(0.)

tensor([0.4929, 1.7259, 0.6689, 0.0000, 1.6048, 0.3592, 0.0000, 0.0000, 0.0000,
        0.0000])

In [25]:
t = relu(lin(x_train,w1,b1))

In [26]:
t.mean(),t.std()

(tensor(0.4432), tensor(0.6143))

Use Kaiming initialization: (read paper)

In [27]:
w1 = torch.randn(tc,nh)*math.sqrt(2/tc);
w2 = torch.randn(nh,1)*math.sqrt(2/nh)

In [28]:
w1.mean(), w1.std()

(tensor(-0.0004), tensor(0.0505))

In [29]:
t = relu(lin(x_train,w1,b1))

We get better reuslt for standard deviation but mean is still jacked up

In [30]:
t.mean(),t.std()

(tensor(0.4661), tensor(0.7495))

In [31]:
#export
from torch.nn import init

Okay, so now we can just use the pytorch version of kaiming initializtion...

In [32]:
w1 = torch.zeros(tc,nh)
init.kaiming_normal_(w1,mode='fan_out')
w2 = torch.zeros(nh,1)
init.kaiming_normal_(w2,mode='fan_out')
t = relu(lin(x_train,w1,b1))

In [33]:
t.mean(),t.std()

(tensor(0.5938), tensor(0.8625))

In [34]:
def relu(x): return x.clamp_min(0.) - .5

In [35]:
t1 = relu(lin(x_train,w1,b1));
t1.mean(),t1.std()

(tensor(0.0938), tensor(0.8625))

In [36]:
def model(xb):
    l1 = lin(xb,w1,b1)
    l2 = relu(l1)
    l3 = lin(l1,w2,b2)
    return l3

In [37]:
%timeit -n 10 _=model(x_train)

24.4 ms ± 350 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Loss Function: MSE

In [38]:
model(x_train).shape

torch.Size([50000, 1])

In [39]:
#export
def mse(output,targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [40]:
y_train.type()

'torch.LongTensor'

In [41]:
y_valid.type()

'torch.LongTensor'

In [42]:
y_train.type()

'torch.LongTensor'

In [43]:
y_valid.type()

'torch.LongTensor'

In [44]:
y_train,y_valid = y_train.float(),y_valid.float()

In [45]:
preds = model(x_train)

In [46]:
preds.type()

'torch.FloatTensor'

In [47]:
y_train.float()

tensor([5., 0., 4.,  ..., 8., 4., 8.])

In [48]:
mse(preds,y_train)

tensor(27.8528)

Alright now I need to have some gradients....

In [49]:
(2*(preds.squeeze(-1) - y_train) / preds.shape[0]).sum()/preds.shape[0]

tensor(-0.0002)

In [50]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [51]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

In [52]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [53]:
def forward_and_backward(inp,targ):
    #forward
    l1 = lin(inp,w1,b1)
    l2 = relu(l1)
    l3 = lin(l2,w2,b2)
    loss = mse(l3,targ)
    
    #backward
    mse_grad(l3,targ)
    lin_grad(l2,l3,w2,b2)
    relu_grad(l1,l2)
    lin_grad(inp,l1,w1,b1)
    
    return l3.shape

In [54]:
forward_and_backward(x_train,y_train)

torch.Size([50000, 1])

In [55]:
x_train

tensor([[-0.4244, -0.4244, -0.4244,  ..., -0.4244, -0.4244, -0.4244],
        [-0.4244, -0.4244, -0.4244,  ..., -0.4244, -0.4244, -0.4244],
        [-0.4244, -0.4244, -0.4244,  ..., -0.4244, -0.4244, -0.4244],
        ...,
        [-0.4244, -0.4244, -0.4244,  ..., -0.4244, -0.4244, -0.4244],
        [-0.4244, -0.4244, -0.4244,  ..., -0.4244, -0.4244, -0.4244],
        [-0.4244, -0.4244, -0.4244,  ..., -0.4244, -0.4244, -0.4244]])

In [56]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

In [57]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [58]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [59]:
loss = forward(xt2, y_train)

In [60]:
loss.backward()

In [61]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

Let's try to clean this up with classes..whooohooo! This should be fun

In [62]:
class MSE():
    def __call__(self,inp,targ):
        self.inp = inp
        self.targ = targ
        self.out = (output.squeeze(-1) - targ).pow(2).mean()
        
        return self.out
    def backward(self): self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.inp.shape[0]

In [63]:
class Lin():
    def __init__(self,w,b):
        self.w = w
        self.b = b
        
    def __call__(self,inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [64]:
class Relu():
    def __call__(self,inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    def backward(self):
        self.inp.g = (self.inp>0).float() * self.out.g

In [65]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = MSE()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

Let's swerve into classes adding attributes to other classes...

In [66]:
class Animals():
    def __call__(self,color):
        self.color = color
        self.out = "hello"
        return self.out

    def do_something(self):
        self.color.g = 3 + self.out.g
        print(f"This is the result: {self.color.g}")
        
    def __str__(self):
        return unicode(self).encode('utf-8')

In [67]:
class BeforeAnimals():
    def __call__(self,inp):
        self.inp = inp
        self.out = "poop"
        return self.out

    def do_something(self): self.inp.g = 2
        
    def __str__(self):
        return unicode(self).encode('utf-8')

Nevermind that BS...I guess I am okay with these classes adding an attribute .g to the tensor objects w1,w2,b1,b2,inp.g

### nn.Linear and nn.Modulen.Module

In [68]:
#export
from torch import nn

## export

In [69]:
!./notebook2script.py 02_fully_connected_OMO.ipynb

Converted 02_fully_connected_OMO.ipynb to exp/nb_02.py
